In [2]:
import tensorflow as tf

# Appréhender le fonctionnement de TF

In [9]:
# Créer des Constantes (Noeuds n'acceptant pas d'entrée)
node1 = tf.constant(3.0, dtype=tf.float32) # tf.constant initialise le noeud
node2 = tf.constant(4.0)                   # implicitement un tf.float32
print(node1, node2)

Tensor("Const_12:0", shape=(), dtype=float32) Tensor("Const_13:0", shape=(), dtype=float32)


In [10]:
# Lancer une Session (computationnal graph), encapsulant le contrôle et l'état des tenseurs
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


In [11]:
# Réaliser des Opérations (qui sont aussi des Noeuds)
from __future__ import print_function
node3 = tf.add(node1,node2)
print("node3:", node3)
print("sess.run(node3)", sess.run(node3))

node3: Tensor("Add:0", shape=(), dtype=float32)
sess.run(node3) 7.0


In [13]:
# Créer des Placeholders (variables auxquelles on pourra attribuer des valeurs)
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b # + est un raccourci pour tf.add(a,b)
add_and_triple = adder_node * 3

print(sess.run(adder_node, {a: 3, b: 4.5}))
print(sess.run(adder_node, {a: [1, 3], b: [2, 4]}))
print(sess.run(add_and_triple, {a: 3, b: 4.5}))

7.5
[ 3.  7.]
22.5


In [14]:
# Créer des Variables (dont les sorties provoquées par une même entrée peuvent être différentes)
W = tf.Variable([.3], dtype=tf.float32)   # 0.3 correponds à sa valeur initiale
b = tf.Variable([-0.3], dtype=tf.float32) # tf.Variable n'initialise pas le noeud
x = tf.placeholder(tf.float32)
linear_model = W*x + b

# Pour initialiser des Variables :
init = tf.global_variables_initializer()
sess.run(init)

In [15]:
# Evaluer linear_model pour plusieurs valeurs du Placeholder x
print(sess.run(linear_model, {x: [1, 2, 3, 4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


In [17]:
# Compléter le modèle linéaire
y = tf.placeholder(tf.float32)               # Contiendra les valeurs désirées

# Calculer le coût :
squared_deltas = tf.square(linear_model - y) # (linear - y) renvoit un vecteur contenant toutes les différences, tf.square met au carré chaucune de ces valeurs
loss = tf.reduce_sum(squared_deltas)         # tf.reduce_sum somme toutes les valeurs de Arg1

print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

23.66


In [18]:
# Optimiser manuellement le modèle pour réduire le coût à 0
fixW = tf.assign(W, [-1]) #tf.assign assigne à Arg1 une nouvelle valeur Arg2
fiwb = tf.assign(b, [1])
sess.run([fixW, fiwb])

print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

0.0


In [19]:
# Optimiser automatiquement le modèle en utilisant un Optimiseur (ici gradient descent, le plus simple)
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

sess.run(init)        # réinitialise les valeurs à celles par défaut, incorrectes
for i in range(1000): # Arg1 : nombre d'itérations à l'apprentissage
    sess.run(train, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})
    
print(sess.run([W, b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


## tf.estimator
Library TF haut-niveau simplifiant les mécanismes de ML (réaliser des entrainements et des évaluations, gérer les jeux de données)

In [20]:
import numpy as np
import tensorflow as tf

In [22]:
# Définir la liste de features
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

# Un estimateur permet d'invoquer l'entrainement (fitting) et l'évaluation (inférence)
# Nombreux types d'estimateurs, ici on utilise un régresseur linéaire
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

# Définition des jeux de données (différent pour l'entrainement et l'évaluation)
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7., 0.])
input_fn = tf.estimator.inputs.numpy_input_fn({"x": x_train}, 
                                               y_train, 
                                               batch_size=4,
                                               num_epochs=None,
                                               shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn({"x": x_train}, 
                                                     y_train, 
                                                     batch_size=4,
                                                     num_epochs=1000,
                                                     shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn({"x": x_eval}, 
                                                    y_eval, 
                                                    batch_size=4,
                                                    num_epochs=1000,
                                                    shuffle=False)

# Invocation de l'entrainement (en Arg2 étapes)
estimator.train(input_fn=input_fn, steps=1000)

# Evaluation de l'efficacité du modèle
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics : %r"% train_metrics)
print("eval metrics : %r"% eval_metrics)

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_session_config': None, '_tf_random_seed': None, '_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_model_dir': '/tmp/tmponues3nq', '_is_chief': True, '_num_ps_replicas': 0, '_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_num_worker_replicas': 1, '_save_checkpoints_secs': 600, '_master': '', '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1956b9d4e0>, '_service': None}


INFO:tensorflow:Using config: {'_session_config': None, '_tf_random_seed': None, '_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_model_dir': '/tmp/tmponues3nq', '_is_chief': True, '_num_ps_replicas': 0, '_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_num_worker_replicas': 1, '_save_checkpoints_secs': 600, '_master': '', '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1956b9d4e0>, '_service': None}


INFO:tensorflow:Create CheckpointSaverHook.


...

INFO:tensorflow:Finished evaluation at 2018-01-17-13:33:12


INFO:tensorflow:Finished evaluation at 2018-01-17-13:33:12


INFO:tensorflow:Saving dict for global step 1000: average_loss = 0.00252721, global_step = 1000, loss = 0.0101089


INFO:tensorflow:Saving dict for global step 1000: average_loss = 0.00252721, global_step = 1000, loss = 0.0101089


train metrics : {'average_loss': 6.6248995e-10, 'global_step': 1000, 'loss': 2.6499598e-09}
eval metrics : {'average_loss': 0.0025272148, 'global_step': 1000, 'loss': 0.010108859}


## tf.estimator (modèle personnalisé)
tf.estimator contient un certain nombre de modèles prédéfinis, mais il est aussi possible créer nos propres modèles personnalisés.  
Pour cela, il faut fournir à tf.estimator.Estimator une fonction modele_fn qui décrit à tf.estimator comment évaluer les prédictions, les étapes d'entrainement et la fonction de perte. 

In [23]:
import numpy as np
import tensorflow as tf

In [27]:
# Définir la liste de features
def model_fn(features, labels, mode):
    # Construire un modèle linéaire et prédire les valeurs
    W = tf.get_variable("W", [1], dtype=tf.float64)
    b = tf.get_variable("b", [1], dtype=tf.float64)
    y = W*features['x'] + b
    
    # Calcul perte
    loss = tf.reduce_sum(tf.square(y - labels))
    
    # Entrainement
    global_step = tf.train.get_global_step()
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = tf.group(optimizer.minimize(loss), tf.assign_add(global_step, 1))
    
    #EstimatorSpec rassemble les sous-parties pour les rendre fonctionnelles
    return tf.estimator.EstimatorSpec(mode=mode,
                                     predictions=y,
                                     loss=loss,
                                     train_op=train)

estimator = tf.estimator.Estimator(model_fn=model_fn)

# Définition des jeux de données
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7., 0.])
input_fn = tf.estimator.inputs.numpy_input_fn({"x": x_train},
                                             y_train,
                                             batch_size=4,
                                             num_epochs=None,
                                             shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn({"x": x_train},
                                             y_train,
                                             batch_size=4,
                                             num_epochs=1000,
                                             shuffle=False)

eval_input_fn = tf.estimator.inputs.numpy_input_fn({"x": x_eval},
                                             y_eval,
                                             batch_size=4,
                                             num_epochs=1000,
                                             shuffle=False)

# Entrainement
estimator.train(input_fn=input_fn, steps=1000)

# Evaluation de l'efficacité du modèle
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics : %r"% train_metrics)
print("eval metrics : %r"% eval_metrics)

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_session_config': None, '_tf_random_seed': None, '_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_model_dir': '/tmp/tmps9jjsjn0', '_is_chief': True, '_num_ps_replicas': 0, '_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_num_worker_replicas': 1, '_save_checkpoints_secs': 600, '_master': '', '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1955998d68>, '_service': None}


INFO:tensorflow:Using config: {'_session_config': None, '_tf_random_seed': None, '_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_model_dir': '/tmp/tmps9jjsjn0', '_is_chief': True, '_num_ps_replicas': 0, '_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_num_worker_replicas': 1, '_save_checkpoints_secs': 600, '_master': '', '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1955998d68>, '_service': None}


INFO:tensorflow:Create CheckpointSaverHook.


...

INFO:tensorflow:Finished evaluation at 2018-01-17-14:19:28


INFO:tensorflow:Finished evaluation at 2018-01-17-14:19:28


INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 0.0101014


INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 0.0101014


train metrics : {'global_step': 1000, 'loss': 1.289916e-10}
eval metrics : {'global_step': 1000, 'loss': 0.01010139}


# Utiliser TF sur MNIST

In [29]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MINST_data/", one_hot=True)

Extracting MINST_data/train-images-idx3-ubyte.gz
Extracting MINST_data/train-labels-idx1-ubyte.gz
Extracting MINST_data/t10k-images-idx3-ubyte.gz
Extracting MINST_data/t10k-labels-idx1-ubyte.gz


In [31]:
# Initialisation des paramètres
x = tf.placeholder(tf.float32, [None,784]) # 784 corresponds à la taille de l'array d'une image MNIST aplatie
W = tf.Variable(tf.zeros([784, 10]))       # 10 corresponds au nombre de classes
b = tf.Variable(tf.zeros([10]))

# Définition du modèle (en une seule ligne)
y = tf.nn.softmax(tf.matmul(x, W) + b)     # matmul : matrice multiplication

# Calcul de la cross-entropy
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

# Entrainement
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [32]:
# Lancer le modèle
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

for _ in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

In [34]:
# Evaluer l'efficacité du modèle
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})) # %precision (state-of-the-art models : 0.997)

0.9149


# DeepLearning pour MNIST via TF

In [37]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [38]:
import tensorflow
sess = tf.InteractiveSession()

In [39]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10]) # labels

W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
sess.run(tf.global_variables_initializer())

In [40]:
y = tf.matmul(x,W) + b

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

In [41]:
# Entrainement
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

for _ in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

In [42]:
# Evaluer l'efficacité du modèle
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9191


### Construire un réseau convolutif multicouche